In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
# To add interactions in linear regressions models
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
vf = pd.read_csv("deepvar_video_features.csv", usecols=["FILENAME", "WIDTH", "HEIGHT", "SPATIAL_COMPLEXITY", "TEMPORAL_COMPLEXITY", "CHUNK_COMPLEXITY_VARIATION", "COLOR_COMPLEXITY"])
vf.head()

,FILENAME,WIDTH,HEIGHT,SPATIAL_COMPLEXITY,TEMPORAL_COMPLEXITY,CHUNK_COMPLEXITY_VARIATION,COLOR_COMPLEXITY
0,Animation_1080P-01b3,1920,1080,0.098,0.004,0.017,0.005
1,Animation_1080P-05f8,1920,1080,1.229,0.454,15.962,0.794
2,Animation_1080P-0c4f,1920,1080,1.714,0.136,9.048,0.000
3,Animation_1080P-0cdf,1920,1080,1.330,0.451,3.677,0.181
4,Animation_1080P-18f5,1920,1080,2.452,0.137,3.918,0.329


In [3]:
vf.describe()

,WIDTH,HEIGHT,SPATIAL_COMPLEXITY,TEMPORAL_COMPLEXITY,CHUNK_COMPLEXITY_VARIATION,COLOR_COMPLEXITY
count,1397.000000,1397.000000,1397.000000,1397.000000,1397.00000,1397.000000
mean,1526.392269,900.219041,1.354391,0.353651,4.15589,0.284471
std,1027.919721,540.881707,0.851814,0.226049,4.63931,0.230898
min,202.000000,320.000000,0.008000,0.001000,0.00200,0.000000
25%,640.000000,480.000000,0.713000,0.173000,0.76700,0.113000
50%,1280.000000,720.000000,1.164000,0.321000,2.40900,0.208000
75%,1920.000000,1080.000000,1.860000,0.516000,6.09100,0.413000
max,5120.000000,2560.000000,4.675000,1.286000,40.49000,2.233000


In [4]:
def elapsedtime_to_sec(el):
    tab = el.split(":")
    return float(tab[0])*60+float(tab[1])

# the data folder, see the markdown there for additional explanations
res_dir = "res_ugc/"

# the list of videos names, e.g. Animation_360P-3e40
# we sort the list so we keep the same ids between two launches
v_names = sorted(os.listdir(res_dir)) 

# the list of measurements
listVideo = []

# we add each dataset in the list, converting the time to the right format
# third line asserts that the measures are complete
for v in v_names:
    data = pd.read_table(res_dir+v, delimiter = ',')
    data['etime'] = [*map(elapsedtime_to_sec, data['elapsedtime'])]
    assert data.shape == (201,34), v
    data["FILENAME"] = os.path.splitext(v)[0]
    listVideo.append(data)


In [5]:
print(" We consider ", len(listVideo), " videos")
listVideo[0].head()

 We consider  1397  videos


,configurationID,cabac,ref,deblock,analyse,me,subme,mixed_ref,me_range,trellis,8x8dct,fast_pskip,chroma_qp_offset,bframes,b_pyramid,b_adapt,direct,weightb,open_gop,weightp,scenecut,rc_lookahead,mbtree,qpmax,aq-mode,size,usertime,systemtime,elapsedtime,cpu,frames,fps,kbs,etime,FILENAME
0,1,0,1,0:0:0,0:0,dia,0,0,16,0,0,1,0,0,None,None,None,None,None,0,0,None,0,69,0,403085,7.94,1.36,0:02.14,434,600,375.22,161.07,2.14,Animation_1080P-01b3
1,101,1,2,1:0:0,0x3:0x113,hex,6,1,16,1,1,1,0,8,2,1,spatial,1,0,1,0,60,1,69,1,234157,23.17,1.79,0:03.40,734,600,217.07,93.57,3.40,Animation_1080P-01b3
2,102,1,2,1:0:0,0x3:0x113,hex,6,1,16,1,1,1,0,8,2,1,auto,1,0,1,0,10,1,69,0,159836,18.42,1.66,0:02.71,739,600,293.42,63.87,2.71,Animation_1080P-01b3
3,103,1,2,0:0:0,0x3:0x3,umh,6,1,16,1,1,1,0,8,2,1,spatial,1,0,1,0,10,1,69,0,163586,22.39,1.53,0:02.78,858,600,276.79,65.37,2.78,Animation_1080P-01b3
4,104,1,16,1:0:0,0x3:0x113,hex,6,1,24,1,1,1,-2,3,2,1,auto,1,0,1,40,30,0,69,1,218392,17.40,1.76,0:02.74,699,600,287.79,87.27,2.74,Animation_1080P-01b3


In [6]:
predDimension = "kbs"
v_names_train = np.loadtxt("train_names.csv", dtype= str)

# save names of test inputs
# np.savetxt("../../data/test_names.csv", v_names_test, fmt='%s')
v_names_test = np.loadtxt("test_names.csv", dtype= str)

In [7]:
listVideoTest = [listVideo[i] for i in range(len(listVideo)) if v_names[i] in v_names_test]
assert len(listVideoTest) == len(v_names_test)

best_perfs = [np.min(vid[predDimension]) for vid in listVideoTest]
best_configs = [np.argmin(vid[predDimension]) for vid in listVideoTest]

In [35]:
listFeatures = ["cabac", "ref", "deblock", "analyse", "me", "subme", "mixed_ref", "me_range", "trellis", 
                "8x8dct", "fast_pskip", "chroma_qp_offset", "bframes", "b_pyramid", 
                "b_adapt", "direct", "weightb", "open_gop", "weightp", "scenecut", "rc_lookahead", 
                "mbtree", "qpmax", "aq-mode"]
categorial = ['analyse', 'me', 'direct']
val_config = listVideo[0][listFeatures].replace(to_replace ="None",value='0')
val_config['deblock'] =[int(val[0]) for val in val_config['deblock']]

for col in val_config.columns:
    if col not in categorial:
        arr_col = np.array(val_config[col],int)
#         arr_col = (arr_col-np.mean(arr_col))/(np.std(arr_col)+1e-5)
        val_config[col] = arr_col
    else:
        if col not in [predDimension,'ranking']:
            val_config[col] = [np.where(k==val_config[col].unique())[0][0] for k in val_config[col]]
            arr_col = np.array(val_config[col],int)
#             arr_col = (arr_col-np.mean(arr_col))/(np.std(arr_col)+1e-5)
            val_config[col] = arr_col

In [36]:
# we load the file (in itself an aggregation of datasets)
# the file is available in the data folder, then ugc_meta
# each line is a video, and the columns are the different metrics
# provided by Wang et. al.
meta = pd.read_csv("deepvar_video_features.csv").set_index('FILENAME')
# category is a high-level characterization of the content of the video
# for an example, Sports for a sports video
# you can see more details about different categories 
# and metrics per category in the resources/categories.csv file
# I also recommand to read the Youtube UGC paper to understand why we consider these categories
meta['category']=[str(meta.index[i]).split('_')[0] for i in range(meta.shape[0])]
# a lot of NA, not a big feature importance, seems complicated to compute -> remove NOISE DMOS
del meta['NOISE_DMOS']
# fill NA with zeros
meta = meta.fillna(0)
# create a numeric variable (quanti) to compute the category
# one video has one and only one category (1 to 1 in sql, so we can join the tables)
# again, to do it properly, we should use dummies
# but then we cannot compare directly the importances of the metrics to categories 
cat_tab = pd.Series(meta['category'].values).unique()
meta['video_category'] = [np.where(cat_tab==meta['category'][i])[0][0] for i in range(len(meta['category']))]
# delete the old columns (quali)
del meta['category']
# we normalize the variables, since height mean is about 1000, and complexity about 2
# different scales do not behave correctly with learning algorithms
# COMMENTED
# for col in meta.columns:#[:len(meta.columns)-1]:
#     inter = np.array(meta[col],float)
#     meta[col] = (inter-np.mean(inter))/np.std(inter)
# END COMMENTED
# left join performance groups to the dataset of metrics
# fake groups
groups = np.random.randint(0,3, len(listVideo))
perf = pd.DataFrame({'FILENAME': np.array([v_names[k][:-4] for k in range(len(v_names))]),
              'perf_group' : np.array([k for k in groups])}).set_index('FILENAME')
meta_perf = perf.join(meta)
# print the results
meta_perf

,perf_group,SLEEQ_DMOS,BANDING_DMOS,WIDTH,HEIGHT,SPATIAL_COMPLEXITY,TEMPORAL_COMPLEXITY,CHUNK_COMPLEXITY_VARIATION,COLOR_COMPLEXITY,video_category
FILENAME,,,,,,,,,,
Animation_1080P-01b3,1,0.000,0.521,1920,1080,0.098,0.004,0.017,0.005,0
Animation_1080P-05f8,2,0.198,0.189,1920,1080,1.229,0.454,15.962,0.794,0
Animation_1080P-0c4f,1,0.003,0.094,1920,1080,1.714,0.136,9.048,0.000,0
Animation_1080P-0cdf,0,0.050,0.121,1920,1080,1.330,0.451,3.677,0.181,0
Animation_1080P-18f5,1,0.026,0.094,1920,1080,2.452,0.137,3.918,0.329,0
...,...,...,...,...,...,...,...,...,...,...
Vlog_720P-561e,0,0.000,0.094,1280,720,2.188,0.034,1.128,0.390,14
Vlog_720P-5d08,2,0.000,0.094,732,720,4.128,0.285,2.126,0.248,14
Vlog_720P-60f8,2,0.146,0.179,1280,720,1.554,0.344,2.466,0.250,14


In [37]:
# we separate the list of videos into a training (i.e. offline) set and a test set (i.e. online)
train_ind, test_ind = train_test_split([k for k in range(len(v_names_train))], test_size = 0.25)
# training set indexes
# train_index = [v[:-4] for v in v_names_train]
train_index = [v_names_train[k][:-4] for k in train_ind]
# test set indexes
test_index = [v_names_train[k][:-4] for k in test_ind]
print(len(train_index), len(test_index))

787 263


In [38]:
# we add the input properties
name_col = list(meta_perf.columns)[1:]

# we add the x264 configuration options
for vcc in val_config.columns:
    name_col.append(vcc)

# Then, X (i.e the predicting variables) =  input properties + software configuration options

# we add the variable to predict, i.e. y the bitrate performance distribution
name_col.append("bitrate")

# X length, the number of predicting variables
nb_col = len(name_col)
# the number of configurations
nb_config = 201

# generate the datasets = (X,y)
def gen_dataset(inputs_names):
    # inputs : names of videos
    # output : aggregation of multiple (X,y) for all the videos in the list of names provided in input 
    
    # the final dataset
    res = pd.DataFrame(np.zeros(nb_config*len(inputs_names)*nb_col).reshape(nb_config*len(inputs_names), nb_col))
    res.columns = name_col
    
    # we add the data video per video
    # LINES 6-10 in Algorithm 1
    for i in range(len(inputs_names)):
        # first, we retrieve the name of the video
        video_name = inputs_names[i]
        index_video = np.where(np.array([v[:-4] for v in v_names], str)==video_name)[0][0]
        # we compute the performance, here
        bitrates = listVideo[index_video][predDimension]
        # get the input properties of the video
        video_prop = np.array(meta_perf.loc[video_name][1:], float)
        # compute the avrage value and the standard deviation for the bitrate
        # as we said in the paper, it does not change the order of variable
        # which is a good property
        moy = np.mean(bitrates)
        std = np.std(bitrates)
        # for each configuration, we add the values of the input properties and the configuration options (=X)
        # and the normalized values of bitrates (=y)
        for config_id in range(nb_config):
            val = list(tuple(video_prop) + tuple(val_config.loc[config_id]))
#             val.append((bitrates[config_id]-moy)/std)
            val.append(bitrates[config_id])
            res.loc[i*nb_config+config_id] = val
    return res

# training dataset
training_data = gen_dataset(train_index)

# dimensions of the different sets = a proxy to the measurement cost 
print("Training size : ", training_data.shape[0])

# OFFLINE - Training data
X_train = np.array(training_data.drop(["bitrate"],axis=1), float)
y_train = np.array(training_data["bitrate"], float)

Training size :  158187


In [45]:
# test dataset
test_data = gen_dataset(test_index)

X_test = np.array(test_data.drop(["bitrate"],axis=1), float)
y_test = np.array(test_data["bitrate"], float)

print("Test size : ", test_data.shape[0])

Test size :  52863


In [46]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [47]:
X_train = MinMaxScaler().fit_transform(X_train)
y_train = MinMaxScaler().fit_transform(y_train.reshape(-1, 1))

In [48]:
X_test = MinMaxScaler().fit_transform(X_test)
y_test = MinMaxScaler().fit_transform(y_test.reshape(-1, 1))

In [49]:
# # Parameters from the grid search in the paper
rgr = RandomForestRegressor(max_depth=None, max_features=15, min_samples_leaf=2, n_estimators=100, n_jobs=4)
X_sel = StandardScaler().fit_transform(X_train)
y_sel = StandardScaler().fit_transform(y_train.reshape(-1, 1)).ravel()
rgr.fit(X_sel, y_sel)
# rgr.score(X_test, y_test)
y_pred_train = rgr.predict(X_sel)
y_pred_test = rgr.predict(X_test)
train_acc = mean_squared_error(y_sel, y_pred_train)
val_acc = mean_squared_error(y_test, y_pred_test)
print(f"100%: {train_acc} / {val_acc}")

100%: 0.0023828767022238064 / 0.008762088761235444


In [50]:
from apricot import FacilityLocationSelection, FeatureBasedSelection

for ratio in [0.01, 0.05, 0.1, 0.25, 0.5, 0.75]:
    selector = FeatureBasedSelection(int(ratio*len(X_train)), 
                                     concave_func='sqrt', 
                                     optimizer='two-stage', 
                                     verbose=False)
    X_sel, y_sel = selector.fit_transform(X_train, y_train)

    # TODO Preprocessing / separate for selection (>0) and training
    X_sel = StandardScaler().fit_transform(X_sel)
    y_sel = StandardScaler().fit_transform(y_sel.reshape(-1, 1)).ravel()
    
    rgr = RandomForestRegressor(max_depth=None, max_features=15, min_samples_leaf=2, n_estimators=100, n_jobs=4)
    rgr.fit(X_sel, y_sel)
    y_pred_train = rgr.predict(X_sel)
    y_pred_test = rgr.predict(X_test)
    train_acc = mean_squared_error(y_sel, y_pred_train)
    val_acc = mean_squared_error(y_test, y_pred_test)
    print(f"{ratio*100}%: {train_acc} / {val_acc}")

1.0%: 0.009439987155759798 / 0.2277872725383245
5.0%: 0.005097883617713009 / 0.2815919563711782
10.0%: 0.003132469133593247 / 0.24342485153803828
25.0%: 0.0018292928367765848 / 0.021617797051118888
50.0%: 0.0014729453496622575 / 0.009568589227896336
75.0%: 0.0014535878877825446 / 0.01944467622076498


In [ ]:
from sklearn.model_selection import GridSearchCV
LA_rf = RandomForestRegressor()

grid_search_larf = GridSearchCV(estimator = LA_rf,
                                param_grid = {'n_estimators': [200, 500, 1000],
                                              # we didn't include 1 for min_samples_leaf to avoid overfitting
                                         'min_samples_leaf' : [2, 5, 10],
                                         'max_depth' : [3, 5, 10, None],
                                         'max_features' : [5, 15, 33]},
                                scoring = 'neg_mean_squared_error',
                                verbose = True,
                                n_jobs = 4)

grid_search_larf.fit(X_train, y_train.ravel())

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [53]:
grid_search_larf.best_params_

{'max_depth': None,
 'max_features': 15,
 'min_samples_leaf': 2,
 'n_estimators': 200}